In [ ]:
import pandas as pd
import statsmodels.api as sm
import statistics
#import docplex.mp.model as cplex
import math
import matplotlib.pyplot as plt
import numpy as np
import re
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import lime
from lime import lime_tabular

from sklearn.cross_decomposition import PLSRegression
pd.set_option('chained_assignment',None)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import data
timesData = pd.read_csv('timesData.csv')


timesData2 = pd.read_csv('times2-revised.csv')


timesData=timesData._append(timesData2)

In [ ]:
atr=["world_rank","university_name","teaching","international","research","citations","income","total_score","year"]

timesData=timesData[atr]
timesData=timesData.reset_index(drop=True)

# data cleaning
# timesData.international = timesData.international.str.replace('-', '')
timesData.world_rank = timesData.world_rank.str.replace('=', '')
timesData.world_rank = timesData.world_rank.str.split('-').str[0]
timesData.world_rank = timesData.world_rank.str.split('+').str[0]
timesData.world_rank = timesData.world_rank.str.split('–').str[0]
timesData.world_rank = timesData.world_rank.str.split('—').str[0]#this line is not duplicated
timesData.total_score = timesData.total_score.str.split('—').str[0]
timesData.total_score = timesData.total_score.str.split('–').str[0]


timesData.international = timesData.international.replace('-', np.nan)
timesData.total_score = timesData.total_score.replace('-', np.nan)
timesData.income = timesData.income.replace('-', np.nan)


#fill in NAN value with previous value 
timesData.isnull().any()
timesData =timesData.fillna(method='ffill')
timesData=timesData.reset_index(drop=True)

# convert all data to numerical values 
timesData['teaching'] = timesData['teaching'].astype(float)

timesData['international'] = timesData['international'].astype(float)

timesData['research'] = timesData['research'].astype(float)

timesData['citations'] = timesData['citations'].astype(float)
timesData['income'] = timesData['income'].astype(float)

timesData['total_score'] = timesData['total_score'].astype(float)
timesData['world_rank'] = timesData['world_rank'].astype(int)

# drop colleges with no total scores
noScores = timesData[ timesData['world_rank'] > 198].index
timesData.drop(noScores , inplace=True)

df = timesData[timesData.year == 2016]


In [ ]:
df[1:20]

In [ ]:
# - FUNCTIONS -

# begin create function
def createNeighborhood(collegeIndex, s):
    # function creates a neighborhood of size s around collegeIndex given
    neighborhood = []
    for x in range(s, 0, -1):
        if collegeIndex - x > 0:
            neighborhood.append(collegeIndex - x)
    neighborhood.append(collegeIndex)
    for x in range(1, s + 1):
        if collegeIndex + x < 203:
            neighborhood.append(collegeIndex + x)
    return neighborhood
# end create function
    
# BEGIN OLS FUNCTION
def identify_most_sig_feature_OLS(neighborhood_array, sig_values):
    # Function uses OLS to find mose contributing features. It returns a 2D 
    # array of each feature in order in most contributing to least of size 
    # parameter 'sig_values'
    
    coefficents = []
    
    first = neighborhood_array[0]
    last = neighborhood_array[-1]
    
    X = df.loc[1802 + first : 1801 + last,"teaching":"income"]
    y = df.loc[1802 + first : 1801 + last, "total_score"]

    model = sm.OLS(y, X)
    results = model.fit()
    #append all coeffecients
    coefficents.append([results.params[0], "teaching"])
    coefficents.append([results.params[1], "international"])
    coefficents.append([results.params[2], "research"])
    coefficents.append([results.params[3], "citations"])
    coefficents.append([results.params[4], "income"])
    #sort coefficents from greatest to least
    coefficents.sort(reverse = True) 
    return(coefficents[0:sig_values])
# END OLS FUNCTION


def vip(x, y, model):
    t = model.x_scores_
    w = model.x_weights_
    q = model.y_loadings_

    m, p = x.shape
    _, h = t.shape

    vips = np.zeros((p,))

    s = np.diag(t.T @ t @ q.T @ q).reshape(h, -1)
    total_s = np.sum(s)

    for i in range(p):
        weight = np.array([ (w[i,j] / np.linalg.norm(w[:,j]))**2 for j in range(h) ])
        vips[i] = np.sqrt(p*(s.T @ weight)/total_s)

    return vips

# BEGIN PLS FUNCTION
def identify_most_sig_feature_PLS(neighborhood_array, sig_values):
    # Function uses PLS to find mose contributing features. It returns a 2D 
    # array of each feature in order in most contributing to least of size 
    # parameter 'sig_values'

    coefficents = []
    df.reset_index(inplace = True, drop = True)

    first = neighborhood_array[0]
    last = neighborhood_array[-1]
    
    X = df.loc[first : last,"teaching":"income"]
    y = df.loc[first : last, "world_rank"]
    pls = PLSRegression(n_components= 5) 
    pls.fit(X, y)
 
    
    #append all coeffecients
    con = len(neighborhood_array)
    coefficents.append(pls.coef_[0][0]/con)
    coefficents.append(pls.coef_[0][1]/con)
    coefficents.append(pls.coef_[0][2]/con)
    coefficents.append(pls.coef_[0][3]/con)
    coefficents.append(pls.coef_[0][4]/con)
    #sort coefficents from greatest to least
    #coefficents.sort(reverse = True)
    output=vip(X, y, pls)
    return(output)
    
    
# END PLS FUNCTION

def transitionProbability(startRank, endRank):
    count = 0
    for x in range(2011, 2020):
        df = timesData[timesData.year == x]
#         df.world_rank = df.world_rank.str.replace('=', '')
#         df.world_rank = df.world_rank.str.split('-').str[0]
#         df['world_rank'] = df['world_rank'].astype(int)
        df = df.iloc[startRank-1,1]
    
        df1 = timesData[timesData.year == x+1]
#         df1.world_rank = df1.world_rank.str.replace('=', '')
#         df1.world_rank = df1.world_rank.str.split('-').str[0]
#         df1['world_rank'] = df1['world_rank'].astype(int)
        df1 = df1.iloc[endRank-1,1]
    
        if df == df1:
            count += 1
    pr = count/5.0
    return pr

def transitionProbabilityBetweenNeighborhood(neighborhoodIndex, neighborhoodSizeOne, neighborhoodSizeTwo):
    count = 0
    for x in range(2011, 2020):
        N=createNeighborhood(neighborhoodIndex,neighborhoodSizeOne)
        for n in N:
            for c in range(neighborhoodIndex-neighborhoodSizeTwo, neighborhoodIndex-neighborhoodSizeOne):
                df = timesData[timesData.year == x]
#                 df.world_rank = df.world_rank.str.replace('=', '')
#                 df.world_rank = df.world_rank.str.split('-').str[0]
#                 df['world_rank'] = df['world_rank'].astype(int)
                df = df.iloc[n-1,1]
                
    
                df1 = timesData[timesData.year == x+1]
#                 df1.world_rank = df1.world_rank.str.replace('=', '')
#                 df1.world_rank = df1.world_rank.str.split('-').str[0]
#                 df1['world_rank'] = df1['world_rank'].astype(int)
                df1 = df1.iloc[c-1,1]
                if df == df1:
                    count += 1
    pr = count / (len(N) * 5.0)
    return pr



In [ ]:
#Framework
unis=7  # 7
universityRank = [5,15,25,45,65,85,100] #5,15,25,45,65,85,100  1,2,6,14
numberOfNeighborhood = 3
sizeOfNeighborhoodList = [5,10,15,30,100,150,200]

In [ ]:
tot=[]
j=0
for j in range(0,unis):
    PLS=[]
    for i in range (0, numberOfNeighborhood):
        neighborhood = createNeighborhood(universityRank[j], sizeOfNeighborhoodList[i])
        PLS.append(identify_most_sig_feature_PLS(neighborhood, 5))
    PLS=pd.DataFrame(PLS)
    tot.append(PLS)
    data_pls = pd.concat(tot, axis=1)



In [ ]:
PLS

In [ ]:
data_pls

In [ ]:
           
def identify_most_sig_feature_dctree(neighborhood_array, sig_values):

    coefficents = []
    df.reset_index(inplace = True, drop = True)

    first = neighborhood_array[0]
    last = neighborhood_array[-1]

    X = df.loc[first : last,"teaching":"income"]
    y = df.loc[first : last, "world_rank"]
    #     pls = PLSRegression(n_components= 5) 
    treeregr = DecisionTreeRegressor(max_depth=2)

    treeregr.fit(X, y)

    importance = treeregr.feature_importances_

    return(importance)

In [ ]:
tot=[]
j=0
for j in range(0,unis):
    tree=[]
    for i in range (0, numberOfNeighborhood):
        neighborhood = createNeighborhood(universityRank[j], sizeOfNeighborhoodList[i])
        tree.append(identify_most_sig_feature_dctree(neighborhood, 5))
    tree=pd.DataFrame(tree)
    tot.append(tree)
    data_dct = pd.concat(tot, axis=1)



In [ ]:
tree

In [ ]:
data_dct

In [ ]:
def identify_most_sig_feature_LR(neighborhood_array):
    # Function uses linear regression to find most contributing features. It returns a 2D 
    # array of each feature in order in most contributing to least of size 
    # parameter 'sig_values'
   

    first = neighborhood_array[0]
    last = neighborhood_array[-1]
    
    X = df.loc[first : last,"teaching":"income"]
    y = df.loc[first : last, "world_rank"]     # why rank and not score?
    
   
    LRreg = LinearRegression()
    LRreg.fit(X, y)
    
    # get importance
    importance = LRreg.coef_
    #score=LRreg.score(X, y)
    #print(score)
    
    
    #reproduce this with linear algebra
    N = len(X)
    p = len(X.columns) + 1  # plus one because LinearRegression adds an intercept term

    X_with_intercept = np.empty(shape=(N, p))
    X_with_intercept[:, 0] = 1
    X_with_intercept[:, 1:p] = X.values

    beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y.values
    #print(beta_hat)
    
    #compute standard errors of the parameter estimates
    y_hat = LRreg.predict(X)
    residuals = y.values - y_hat
    residual_sum_of_squares = residuals.T @ residuals
    sigma_squared_hat = residual_sum_of_squares / (N - p)
    var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
    SEV=[]
    for p_ in range(p):
        standard_error = var_beta_hat[p_, p_] ** 0.5
        print(f"SE(beta_hat[{p_}]): {standard_error}")
        SEV.append(standard_error)
    
    #print(SEV)
    standard_error_vector = np.delete(SEV, [0])
    
    t_statistic = (importance/standard_error)
    #print(importance)
    #print(standard_error_vector)
    #print(t_statistic)
  
    return(t_statistic)

In [ ]:
tot=[]
j=0
for j in range(0,unis):
    LR=[]
    for i in range (0, numberOfNeighborhood):
        neighborhood = createNeighborhood(universityRank[j], sizeOfNeighborhoodList[i])
        LR.append(identify_most_sig_feature_LR(neighborhood))
        #print(neighborhood)
    LR=pd.DataFrame(LR)

    tot.append(LR)
    data_LR = pd.concat(tot, axis=1)

In [ ]:
data_LR

In [ ]:
from models.shapley import ShapleyValue

In [ ]:
sv = ShapleyValue(df, [
            'teaching','international','research','citations','income'
        ],
        'total_score'
)

In [ ]:
tot=[]
j=0
atr=['teaching','international','research','citations','income']
for j in range(0,unis):
    SV=[]
    for i in range (0, numberOfNeighborhood):
        neighborhood = createNeighborhood(universityRank[j], sizeOfNeighborhoodList[i])
        sv = ShapleyValue(df.iloc[neighborhood], atr,'world_rank')
        cont=sv.get_shapley_contribution()
        SV.append(cont[atr].values[0])
    SV=pd.DataFrame(SV)
    tot.append(SV)
    data_SV = pd.concat(tot, axis=1)

In [ ]:
data_SV

In [ ]:
def identify_most_sig_feature_Lime(neighborhood,sizeOfNeighborhoodList):

    first = neighborhood[0]
    last = neighborhood[-1]

    X = df.loc[first : last,"teaching":"income"]
    y = df.loc[first : last, "world_rank"]   

    model = LinearRegression()
    model.fit(X, y)
    

    
    # This works with X_Train data
    explainer = lime_tabular.LimeTabularExplainer(
        training_data=np.array(X),
        feature_names=X.columns,
        class_names=['world_rank'],
        mode='regression'
    )
    
    
    ika = sizeOfNeighborhoodList 
    # This works with X_Test data:  Should ika be the university under study???### HA: I think yes but now it is neighborhood size
    exp = explainer.explain_instance(
        data_row=X.iloc[ika],
        predict_fn=model.predict #model.predict_proba
    )
    
    a=exp.as_list()
  

    exp.show_in_notebook(show_table=True)
    
    print('a is :', a)

    #['teaching','international','research','citations','income']
    imp=[0] * 5 
    for i in range(0,5):
        s = a[i][0]
        result = re.search('< (.*) <=', s)
    #     print(result)
        if result==None:
            result = re.search('(.*) >', s)
        if result==None:
            result = re.search('(.*) <=', s)
        if result.group(1)== 'research':
            imp[2]=a[i][1]
        if result.group(1)== 'international':
            imp[1]=a[i][1] 
        if result.group(1)== 'teaching':
            imp[0]=a[i][1]
        if result.group(1)== 'citations':
            imp[3]=a[i][1]
        if result.group(1)== 'income':
            imp[4]=a[i][1]

     # you need to call random.seed before calling explain_instance each time.
     #   I tried setting a seed but for all instances in the j loop it gives the same answer!!
        
 #   def explain(instance, predict_fn,ika):
 #       np.random.seed(16)
 #       exp = explainer.explain_instance(data_row=X.iloc[ika], predict_fn=model.predict)
 #   return exp
 #   a=exp.as_list()



# Prediction_local is intercept + weights * features.
#the weights are relative to the scaled data, so your feature_value column should be scaled. 
#You can get the scaled values by running:
    #b = (X.iloc[ika] - explainer.scaler.mean_) / explainer.scaler.scale_

#TabularExplainer discretizes continuous features by default (discretize_continuous=True in init).    
#https://github.com/marcotcr/lime/issues/189
    
    
    
 #   for runner in exp.as_list():
  #      print(runner[0], "\t", runner[1])

    return(imp)


In [ ]:
l=[]
tot=[]
j=0
for j in range(0,unis):
    LR=[]
    for i in range (0, numberOfNeighborhood):
        neighborhood = createNeighborhood(universityRank[j], sizeOfNeighborhoodList[i])
        a = identify_most_sig_feature_Lime(neighborhood,sizeOfNeighborhoodList[i])
        print(a)
        LR.append(a)
    LR=pd.DataFrame(LR)

    tot.append(LR)
    data_Lime = pd.concat(tot, axis=1)    
        

In [ ]:
data_Lime

In [ ]:
import re
#['teaching','international','research','citations','income']
imp=[0] * 5 
for i in range(0,5):
    s = a[i][0]
    result = re.search('< (.*) <=', s)
#     print(result)
    if result==None:
        result = re.search('(.*) >', s)
    if result==None:
        result = re.search('(.*) <=', s)
    if result.group(1)== 'research':
        imp[2]=a[i][1]
    if result.group(1)== 'international':
        imp[1]=a[i][1] 
    if result.group(1)== 'teaching':
        imp[0]=a[i][1]
    if result.group(1)== 'citations':
        imp[3]=a[i][1]
    if result.group(1)== 'income':
        imp[4]=a[i][1]

In [ ]:
###heatmap for OLS
from mpl_toolkits.axes_grid.parasite_axes import SubplotHost
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 24})
from mpl_toolkits.axes_grid1 import make_axes_locatable
# fig, ax = plt.subplots(1,1,figsize=(20,7.5))
fig1 = plt.figure(figsize=(20,5))
ax1 = SubplotHost(fig1, 111)
fig1.add_subplot(ax1)

ax1.imshow(data_ols, cmap=plt.cm.get_cmap('Blues', 6), interpolation='nearest',aspect='auto')
# ax.colorbar(extend='both')
img = ax1.imshow(data_ols, cmap=plt.cm.get_cmap('Blues', 6), interpolation='nearest',aspect='auto')

x_label_list = [r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$']
y_label_list = ['$\mathcal{H}_1$','$\mathcal{H}_2$','$\mathcal{H}_3$']
ax1.set_xticks(np.arange(0,35))
ax1.set_yticks([  0,  1,  2])
ax1.set_xticklabels(x_label_list)
ax1.set_yticklabels(y_label_list)
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
# Second X-axis
ax2 = ax1.twiny()
offset = 0, -35 # Position of the second axis
new_axisline = ax2.get_grid_helper().new_fixed_axis
ax2.axis["bottom"] = new_axisline(loc="bottom", axes=ax2, offset=offset)
ax2.axis["top"].set_visible(False)

ax2.set_xticks([ 0, 5, 10, 15, 20, 25, 30, 35])
ax2.xaxis.set_major_formatter(ticker.NullFormatter())
ax2.xaxis.set_minor_locator(ticker.FixedLocator([2, 7,12,17,22,27,32]))
ax2.xaxis.set_minor_formatter(ticker.FixedFormatter(['Uni.52', 'Uni.55','Uni.57', 'Uni.60','Uni.62','Uni.65','Uni.71']))
fig1.colorbar(img,cax=cax)
plt.show() 5,15,25,45,65,85,100



In [ ]:
###heatmap for PLS
from mpl_toolkits.axes_grid.parasite_axes import SubplotHost
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 24})
from mpl_toolkits.axes_grid1 import make_axes_locatable
# fig, ax = plt.subplots(1,1,figsize=(20,7.5))
fig1 = plt.figure(figsize=(20,5))
ax1 = SubplotHost(fig1, 111)
fig1.add_subplot(ax1)

ax1.imshow(data_pls, cmap=plt.cm.get_cmap('Blues', 6), interpolation='nearest',aspect='auto')
# ax.colorbar(extend='both')
img = ax1.imshow(data_pls, cmap=plt.cm.get_cmap('Blues', 6), interpolation='nearest',aspect='auto')

x_label_list = [r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$']
y_label_list = ['$\mathcal{H}_1$','$\mathcal{H}_2$','$\mathcal{H}_3$']
ax1.set_xticks(np.arange(0,35))
ax1.set_yticks([  0,  1,  2])
ax1.set_xticklabels(x_label_list)
ax1.set_yticklabels(y_label_list)
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
# Second X-axis
ax2 = ax1.twiny()
offset = 0, -35 # Position of the second axis
new_axisline = ax2.get_grid_helper().new_fixed_axis
ax2.axis["bottom"] = new_axisline(loc="bottom", axes=ax2, offset=offset)
ax2.axis["top"].set_visible(False)

ax2.set_xticks([ 0, 5, 10, 15, 20, 25, 30, 35])
ax2.xaxis.set_major_formatter(ticker.NullFormatter())
ax2.xaxis.set_minor_locator(ticker.FixedLocator([2, 7,12,17,22,27,32]))
ax2.xaxis.set_minor_formatter(ticker.FixedFormatter(['Uni.4', 'Uni.7','Uni.13', 'Uni.17','Uni.62','Uni.65','Uni.71']))
fig1.colorbar(img,cax=cax)
plt.show() 5,15,25,45,65,85,100



In [ ]:
###heatmap for Decision tree
from mpl_toolkits.axes_grid.parasite_axes import SubplotHost
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 24})
from mpl_toolkits.axes_grid1 import make_axes_locatable
# fig, ax = plt.subplots(1,1,figsize=(20,7.5))
fig1 = plt.figure(figsize=(20,5))
ax1 = SubplotHost(fig1, 111)
fig1.add_subplot(ax1)

ax1.imshow(data_dct, cmap=plt.cm.get_cmap('Blues', 6), interpolation='nearest',aspect='auto')
# ax.colorbar(extend='both')
img = ax1.imshow(data_dct, cmap=plt.cm.get_cmap('Blues', 6), interpolation='nearest',aspect='auto')

x_label_list = [r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$']
y_label_list = ['$\mathcal{H}_1$','$\mathcal{H}_2$','$\mathcal{H}_3$']
ax1.set_xticks(np.arange(0,35))
ax1.set_yticks([  0,  1,  2])
ax1.set_xticklabels(x_label_list)
ax1.set_yticklabels(y_label_list)
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
# Second X-axis
ax2 = ax1.twiny()
offset = 0, -35 # Position of the second axis
new_axisline = ax2.get_grid_helper().new_fixed_axis
ax2.axis["bottom"] = new_axisline(loc="bottom", axes=ax2, offset=offset)
ax2.axis["top"].set_visible(False)

ax2.set_xticks([ 0, 5, 10, 15, 20, 25, 30, 35])
ax2.xaxis.set_major_formatter(ticker.NullFormatter())
ax2.xaxis.set_minor_locator(ticker.FixedLocator([2, 7,12,17,22,27,32]))
ax2.xaxis.set_minor_formatter(ticker.FixedFormatter(['Uni.4', 'Uni.7','Uni.13', 'Uni.17','Uni.62','Uni.65','Uni.71']))
fig1.colorbar(img,cax=cax)
plt.show()


In [ ]:
###heatmap for SHAP
from mpl_toolkits.axes_grid.parasite_axes import SubplotHost
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 24})
from mpl_toolkits.axes_grid1 import make_axes_locatable
# fig, ax = plt.subplots(1,1,figsize=(20,7.5))
fig1 = plt.figure(figsize=(20,5))
ax1 = SubplotHost(fig1, 111)
fig1.add_subplot(ax1)

ax1.imshow(data_SV, cmap=plt.cm.get_cmap('Blues', 6), interpolation='nearest',aspect='auto')
# ax.colorbar(extend='both')
img = ax1.imshow(data_SV, cmap=plt.cm.get_cmap('Blues', 6), interpolation='nearest',aspect='auto')

x_label_list = [r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$']
y_label_list = ['$\mathcal{H}_1$','$\mathcal{H}_2$','$\mathcal{H}_3$']
ax1.set_xticks(np.arange(0,35))
ax1.set_yticks([  0,  1,  2])
ax1.set_xticklabels(x_label_list)
ax1.set_yticklabels(y_label_list)
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
# Second X-axis
ax2 = ax1.twiny()
offset = 0, -55 # Position of the second axis
new_axisline = ax2.get_grid_helper().new_fixed_axis
ax2.axis["bottom"] = new_axisline(loc="bottom", axes=ax2, offset=offset)
ax2.axis["top"].set_visible(False)

ax2.set_xticks([ 0, 5, 10, 15, 20, 25, 30, 35])
ax2.xaxis.set_major_formatter(ticker.NullFormatter())
ax2.xaxis.set_minor_locator(ticker.FixedLocator([2, 7,12,17,22,27,32]))
ax2.xaxis.set_minor_formatter(ticker.FixedFormatter(['Uni.4', 'Uni.7','Uni.13', 'Uni.17','Uni.62','Uni.65','Uni.71']))
fig1.colorbar(img,cax=cax)
plt.show()



In [ ]:
###heatmap for Linear Regression
from mpl_toolkits.axes_grid.parasite_axes import SubplotHost
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 24})
from mpl_toolkits.axes_grid1 import make_axes_locatable
# fig, ax = plt.subplots(1,1,figsize=(20,7.5))
fig1 = plt.figure(figsize=(20,5))
ax1 = SubplotHost(fig1, 111)
fig1.add_subplot(ax1)

ax1.imshow(data_LR, cmap=plt.cm.get_cmap('Blues', 6), interpolation='nearest',aspect='auto')
# ax.colorbar(extend='both')
img = ax1.imshow(data_LR, cmap=plt.cm.get_cmap('Blues', 6), interpolation='nearest',aspect='auto')

x_label_list = [r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$', 
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$',
               r'$X_1$', r'$X_2$',r'$X_3$',r'$X_4$',r'$X_5$']
y_label_list = ['$\mathcal{H}_1$','$\mathcal{H}_2$','$\mathcal{H}_3$']
ax1.set_xticks(np.arange(0,35))
ax1.set_yticks([  0,  1,  2])
ax1.set_xticklabels(x_label_list)
ax1.set_yticklabels(y_label_list)
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
# Second X-axis
ax2 = ax1.twiny()
offset = 0, -35 # Position of the second axis
new_axisline = ax2.get_grid_helper().new_fixed_axis
ax2.axis["bottom"] = new_axisline(loc="bottom", axes=ax2, offset=offset)
ax2.axis["top"].set_visible(False)

ax2.set_xticks([ 0, 5, 10, 15, 20, 25, 30, 35])
ax2.xaxis.set_major_formatter(ticker.NullFormatter())
ax2.xaxis.set_minor_locator(ticker.FixedLocator([2, 7,12,17,22,27,32]))
ax2.xaxis.set_minor_formatter(ticker.FixedFormatter(['Uni.4', 'Uni.7','Uni.13', 'Uni.17','Uni.62','Uni.65','Uni.71']))
fig1.colorbar(img,cax=cax)
plt.show()

